In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1439, with_units=False)
Hist_table


Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"08s 19 ; 14,47"
730 ;,"05s 08 ; 29,33"
1095 ;,"01s 27 ; 44,20"
1461 ;,"11s 11 ; 21,59"
1826 ;,"08s 00 ; 36,46"
2191 ;,"04s 19 ; 51,32"
2556 ;,"01s 09 ; 06,19"
2922 ;,"10s 22 ; 43,59"
3287 ;,"07s 11 ; 15,45"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(34):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=2*Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(34):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=7):
        days=Hist_table[i][0]
        Test_recomp[i][1]=(daily_mean*days)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])

In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(985.0, nan)

In [6]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Years,
365 ;,0.000000
730 ;,0.000000
1095 ;,0.000000
1461 ;,0.000000
1826 ;,0.000000
2191 ;,0.000000
2556 ;,0.000000
2922 ;,0.000000
3287 ;,-2580.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar mean center years.csv')
B.to_csv('Lunar mean center years.zip', index=False, compression=compression_opts)